In [1]:
import sys
import re
import glob
from io_functions import *
from nn_packages import  *
from __future__ import print_function
import h5py
import numpy as np
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN not available)

Couldn't import dot_parser, loading of dot files will not be possible.


Welcome to ROOTaaS 6.06/04


In [ ]:
#Class to make generator returning both energy and label
class My_Gen_EL:
    def __init__( self, batch_size, filesize, filepattern='/data/shared/LCD/EnergyScan_Gamma_Shuffled/GammaEscan_*_*.h5'):
        self.batch_size = batch_size
        self.filelist = filter(None, os.popen('ls %s'%filepattern).read().split('\n'))
        #You can change the fraction of train, validation and test set here
        self.train_split = 0.6 
        self.test_split = 0.2 
        self.validation_split = 0.2
        self.fileindex = 0
        self.filesize = filesize
        self.position = 0
    #function to call when generating data for training  
    def train(self,cnn=False):
        length = len(self.filelist)
        #deleting the validation and test set filenames from the filelist
        del self.filelist[np.floor((1-(self.train_split))*length).astype(int):]
        return self.batches(cnn)
    #function to call when generating data for testing
    def test(self, cnn=False):
        length = len(self.filelist)
        #deleting the train and validation set filenames from the filelist
        del self.filelist[:np.floor((1-self.test_split)*length).astype(int)+1]
        return self.batches(cnn)
    #function to call when generating data for validating
    def validation(self, cnn=False):
        length = len(self.filelist)
        #modifying the filename list to only include files for validation set
        self.filelist = self.filelist[np.floor(self.train_split*length+1).astype(int):np.floor((self.train_split+self.validation_split)*length+1).astype(int)]
        return self.batches(cnn)
    #The function which reads files and returns data of batch size of N
    def batch_helper(self, fileindex, position, batch_size, train=True):
        '''
        Yields batches of data of size N
        '''
        f = h5py.File(self.filelist[fileindex],'r')
        #If the data to be read can be read from the current file
        if (position + batch_size < self.filesize):
            data = np.array(f['images'][position : position + batch_size])
            target = np.array(f['target'][position : position + batch_size])
            #incrementing the position to start from while reading the next batch
            position += batch_size
            f.close()
            
            return data, target, fileindex, position
        
        else:
        #if the data to be read exceeds the current file
        #Read the data as much as we can from the current file
            data = np.array(f['images'][position:])
            target = np.array(f['target'][position:])
            f.close()
            #Read the remaining data from the next files by calling the same function recursively
            #Also a safety check to see if the file opened is the last file
            if (fileindex+1 < len(self.filelist)):
                data_, target_, fileindex, position = self.batch_helper(fileindex + 1, 0, batch_size - self.filesize + position)
                data = np.concatenate((data, data_), axis=0)
                target = np.concatenate((target, target_), axis=0)
            #if the file read is the last file, loop back to the beginning of the filname list
            else:
                fileindex = 0
                position = 0
            
            return data, target, fileindex, position
    #The function which loops indefinitely and continues to return data of the specified batch size
    def batches(self, cnn):
        #loop indefinetly
        while (self.fileindex < len(self.filelist)):
            data, target, self.fileindex, self.position = self.batch_helper(self.fileindex, self.position, self.batch_size)
            if data.shape[0]!=self.batch_size:
                continue
            if cnn==True:
                data = np.swapaxes(data,1,3)
            else:
                data= np.reshape(data,(self.batch_size,-1))
            target0=target[:,0]
            target1=target[:,1]
            yield (data, [target[:,0], target[:,1]/110.])
        self.fileindex = 0
        

In [2]:
#Class to make generator returning energy 
class My_Gen_E:
    def __init__( self, batch_size, filesize, filepattern='/data/kaustuv1993/EnergyScan_Gamma/GammaEscan_*_shuffled.h5'):
        self.batch_size = batch_size
        self.filelist = filter(None, os.popen('ls %s'%filepattern).read().split('\n'))
        
        self.train_split = 0.6 
        self.test_split = 0.2 
        self.validation_split = 0.2
        self.fileindex = 0
        self.filesize = filesize
        self.position = 0
    #function to call when generating data for training  
    def train(self, cnn=False):
        length = len(self.filelist)
        del self.filelist[np.floor((1-(self.train_split))*length).astype(int):]
        return self.batches(cnn)
    #function to call when generating data for testing  
    def test(self, cnn=False):
        length = len(self.filelist)
        del self.filelist[:np.floor((1-self.test_split)*length).astype(int)+1]
        return self.batches(cnn)
    #function to call when generating data for validation 
    def validation(self, cnn=False):
        length = len(self.filelist)
        self.filelist = self.filelist[np.floor(self.train_split*length+1).astype(int):np.floor((self.train_split+self.validation_split)*length+1).astype(int)]
        return self.batches(cnn)
        
    #The function which reads files to gather data until batch size is satisfied
    def batch_helper(self, fileindex, position, batch_size):
        '''
        Yields batches of data of size N
        '''
        f = h5py.File(self.filelist[fileindex],'r')
        if (position + batch_size < self.filesize):
            data = np.array(f['images'][position : position + batch_size])
            target = np.array(f['target'][position : position + batch_size])
            target = np.delete(target,0,1)

            position += batch_size
            f.close()
            
            return data, target, fileindex, position
        
        else:
            data = np.array(f['images'][position:])
            target = np.array(f['target'][position:])
            target = np.delete(target,0,1)
            f.close()
            
            if (fileindex+1 < len(self.filelist)):
                data_, target_, fileindex, position = self.batch_helper(fileindex + 1, 0, batch_size - self.filesize + position)
                data = np.concatenate((data, data_), axis=0)
                target = np.concatenate((target, target_), axis=0)
            
            else:
                fileindex = 0
                position = 0
            
            return data, target, fileindex, position
    #The function which loops indefinitely and continues to return data of the specified batch size
    def batches(self, cnn):
        while (self.fileindex < len(self.filelist)):
            data, target, self.fileindex, self.position = self.batch_helper(self.fileindex, self.position, self.batch_size)
            if data.shape[0]!=self.batch_size:
                continue
            if cnn==True:
                data = np.swapaxes(data,1,3)
            else:
                data= np.reshape(data,(self.batch_size,-1))
            yield (data, target/110.)
        self.fileindex = 0
        

In [5]:
ds = My_Gen_E(10000, 10000)
#vs = My_Gen_E(10000,10000)
#count = 0
#print(len(ds.filelist))
for (data, target) in ds.train():
    #print('count is ',count)
    #code to check for unique rows in the data
    ncols = data.shape[1]
    dtype = data.dtype.descr * ncols
    struct = data.view(dtype)
    uniq = np.unique(struct)
    uniq = uniq.view(data.dtype).reshape(-1, ncols)
    #code to check for the number of unique rows (energy) in targets
    count2 = np.unique(target, return_counts=True)[1]
    print(uniq.shape[0],count2.shape[0])

9962 100
9961 100
9979 100
9975 99
9952 99
9976 92


KeyboardInterrupt: 

In [ ]:
simple = Sequential()
simple.add(Dense(1000, input_dim=(10000), activation='relu'))
simple.add(Dense(100, activation='relu'))
simple.add(Dense(1, activation='sigmoid'))
simple.compile(loss='mae', optimizer='sgd')
#simple.load_weights('first_try.h5')
#simple.summary()



In [ ]:
model = Sequential()
model.add(Convolution2D(10, 4, 4, input_shape = (25, 20, 20), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())

# Dense layer
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='mse', optimizer='sgd')

model.summary()

In [ ]:
hist = simple.fit_generator(ds.train(), samples_per_epoch=10000, nb_epoch=5, validation_data= vs.validation(), nb_val_samples=5000,verbose=2)

In [ ]:
model_name = "cnn2d_dense_4_mse_sgd_1"
cnn1.summary()
cnn1.save_weights('%s.h5'%model_name, overwrite=True)
model_json = cnn1.to_json()
with open("%s.json"%model_name, "w") as json_file:
    json_file.write(model_json)

In [ ]:
simple.save_weights('conv_relu10000relu100sigmoid1epoch50adam_mae_mse.h5')

In [ ]:
json_file = open('/home/kaustuv1993/Notebooks/models/cnn2d_dense_4_mse_sgd_300epoch.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.compile(loss='mse', optimizer='sgd')

In [ ]:
model.load_weights('/home/kaustuv1993/Notebooks/weights/cnn2d_dense_4_mse_sgd_300epoch.h5')

In [ ]:
#matplotlib inline
import matplotlib.pyplot as plt

def show_losses( histories, fname ):
    plt.figure(figsize=(10,10))
    #plt.ylim(bottom=0)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Error by Epoch')
    
    colors=[]
    do_acc=False
    for label,loss in histories:
        color = tuple(np.random.random(3))
        colors.append(color)
        l = label
        vl= label+" validation"
        if 'acc' in loss.history:
            l+=' (acc %2.4f)'% (loss.history['acc'][-1])
            do_acc = True
        if 'val_acc' in loss.history:
            vl+=' (acc %2.4f)'% (loss.history['val_acc'][-1])
            do_acc = True
        plt.plot(loss.history['loss'], label=l, color=color)
        if 'val_loss' in loss.history:
            plt.plot(loss.history['val_loss'], lw=2, ls='dashed', label=vl, color=color)


    plt.legend()
    plt.yscale('log')
    plt.show()
    plt.savefig('%s.png'%fname)
    plt.savefig('%s.pdf'%fname)
    if not do_acc: 
	return

    plt.figure(figsize=(10,10))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    for i,(label,loss) in enumerate(histories):
        color = colors[i]
        if 'acc' in loss.history:
            plt.plot(loss.history['acc'], lw=2, label=label+" accuracy", color=color)
        if 'val_acc' in loss.history:
            plt.plot(loss.history['val_acc'], lw=2, ls='dashed', label=label+" validation accuracy", color=color)
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
show_losses([("mse",hist)],'not_important')

In [ ]:
def test_per_energy():
    average_mse = []
    for i in xrange(10,110):
        print (i)
        fn =('/data/kaustuv1993/EnergyScan_Gamma/GammaEscan_%d_shuffled.h5'%i)
        f = h5py.File(fn,'r')
        test_data = np.array(f['images'])
        test_target=np.array(f['target'])
        test_data.shape
        test_data = np.swapaxes(test_data,1,3)
        test_target = np.delete(test_target,0,1)
        test_target.shape
        
        #pred = model.predict(test_data)
        #print([test_target])
        #plt.hist( np.ravel(pred[1]*110)/test_target[:] , label='energy resolution', bins=40)
        #plt.legend()
        #plt.show()
        #print(pred)
        #average_mse.append(pred)
    #print (average_mse)
        



In [ ]:
test_per_energy()

In [ ]:
#unique, counts = np.unique(test_target[:,1], return_counts=True)
        #print (fn, unique, counts)
        #print(fn)
        #print(test_target[:,1].T)   

In [ ]:
#functional Keras model with one hidden layer
#from keras.models import Model
#from keras.layers import Input, Dense, Dropout
#i = Input(shape=(10000,))
#d = Dense(1000, activation='sigmoid')(i)
#dp = Dropout(0.5)(d)
#one output layer with two output neurons, one for classification and another for regression
#oc = Dense(1,activation='sigmoid', name='label')(d)
#oe = Dense(1,activation='sigmoid', name='energy')(d)

#m_model = Model(input=i, output=[oc,oe])
#m_model.compile(loss=['binary_crossentropy','mse'], optimizer='sgd')
#Getting the data and training the net
#hist = m_model.fit_generator(ds.batches(), samples_per_epoch=1800000, nb_epoch = 2, verbose=1)

In [ ]:
pred = simple.evaluate_generator(ds.test(), val_samples=10000)

In [ ]:
print (pred)

In [ ]:
import numpy as np
x = np.array([1,1,1,2,2,2,5,25,1,1])
y = np.bincount(x)
y

In [ ]:
import numpy as np

x = np.array([1,1,1,2,2,2,5,25,1,1])
unique, counts = np.unique(x, return_counts=True)
print (x)
print (unique, counts)
print (np.argmin(counts), counts, counts.min())
current = np.split(x,np.cumsum(counts))
current

In [ ]:
a = np.arange(10,120,10)
a

In [ ]:
def test_per_energy(simple):
    average_mse = []
    for i in xrange(10,110):
        print (i)
        fn =('/data/kaustuv1993/EnergyScan_Gamma/GammaEscan_%dGeV_fulldataset.h5'%i)
        f = h5py.File(fn,'r')
        test_data = np.array(f['images'])
        test_target=np.array(f['target'])
        #test_data = np.swapaxes(test_data,1,3)
        test_data = test_data.reshape(test_data.shape[0],-1)
        test_target = np.delete(test_target,0,1)
        pred = simple.predict(test_data)
        print (pred[:6], test_target[:6]/110.)
        #print([test_target])
        #plt.hist( test_target - (np.ravel(pred[1]*110))/test_target[:] , label='energy resolution', bins=40)
        #plt.legend()
        #plt.show()
        #print(pred)
        #average_mse.append(pred)
    #print (average_mse)
        



In [ ]:
test_per_energy(simple)

In [ ]:
import numpy as np
rperm=np.random.permutation(3)
print (rperm)

In [8]:
a = np.arange(64)
a = a.reshape((4,4,4))
a

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]],

       [[16, 17, 18, 19],
        [20, 21, 22, 23],
        [24, 25, 26, 27],
        [28, 29, 30, 31]],

       [[32, 33, 34, 35],
        [36, 37, 38, 39],
        [40, 41, 42, 43],
        [44, 45, 46, 47]],

       [[48, 49, 50, 51],
        [52, 53, 54, 55],
        [56, 57, 58, 59],
        [60, 61, 62, 63]]])

In [10]:
a[2:5]

array([[[32, 33, 34, 35],
        [36, 37, 38, 39],
        [40, 41, 42, 43],
        [44, 45, 46, 47]],

       [[48, 49, 50, 51],
        [52, 53, 54, 55],
        [56, 57, 58, 59],
        [60, 61, 62, 63]]])

In [25]:
filepattern='/data/shared/LCD/EnergyScan_Gamma_Shuffled/GammaEscan_*GeV_fulldataset.h5'
filelist = filter(None, os.popen('ls %s'%filepattern).read().split('\n'))
print (filelist[98])
f = h5py.File(filelist[98],'r')
data = np.array(f['images'][0 : 10000])
target = np.array(f['target'][0 : 10000])
target = np.delete(target,0,1)
u = np.unique(target)
print(data.shape, u.shape[0])


/data/shared/LCD/EnergyScan_Gamma_Shuffled/GammaEscan_99GeV_fulldataset.h5
(10000, 20, 20, 25) 85
